In [1]:
from cohortshapley import dataset
from cohortshapley import realistic
from cohortshapley import sampling
from cohortshapley import similarity
from cohortshapley import baselineshapley as bs
from cohortshapley import cohortshapley as cs
from cohortshapley import similarity
from cohortshapley import figure
from cohortshapley import varianceshapley as vs
from cohortshapley import cohortowen as co
import numpy as np

In [2]:
X,Y,categorical = dataset.titanic()

In [3]:
from sklearn.linear_model import LogisticRegression

def simple_logistic_regression(X,Y):
    # generate prediction model using Logistic Regression
    model = LogisticRegression(random_state=0).fit(X.values, Y.flatten())
    f = lambda x: model.predict_proba(x).T[1].reshape(-1,1)
    return model, f

In [4]:
model, f = simple_logistic_regression(X,Y)

In [5]:
subject_id = list(range(len(X.index)))
subject = X.values[subject_id]

In [6]:
# set distance threashold for similarity function
similarity.ratio=0.1
# compute cohort Shapley value
cs_obj = cs.CohortShapley(f, similarity.similar_in_distance_cutoff, subject_id, X.values, parallel=4)
cs_obj.compute_cohort_shapley()

use given model to predict y.
parallel processing with 4 processes
use given y values instead of model prediction.use given y values instead of model prediction.
use given y values instead of model prediction.
use given y values instead of model prediction.

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

100%|██████████| 261/261 [00:51<00:00,  5.12it/s]


In [7]:
X.columns

Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare'], dtype='object')

In [8]:
def extractDigits(lst):
    return list(map(lambda el:[el], lst))

In [9]:
#three union structures, the first two should match cohort shapley
titanic_union_structure1 = extractDigits(range(6))
titanic_union_structure2 = [list(range(6))]
titanic_union_structure3 = [[0,5],[3,4],[1],[2]]

In [10]:
print(titanic_union_structure1)
print(titanic_union_structure2)

[[0], [1], [2], [3], [4], [5]]
[[0, 1, 2, 3, 4, 5]]


In [11]:
co_obj1 = co.CohortOwen(titanic_union_structure1,f, similarity.similar_in_distance_cutoff, subject_id, X.values, parallel=4)
co_obj2 = co.CohortOwen(titanic_union_structure2,f, similarity.similar_in_distance_cutoff, subject_id, X.values, parallel=4)
co_obj3 = co.CohortOwen(titanic_union_structure3,f, similarity.similar_in_distance_cutoff, subject_id, X.values, parallel=4)

use given model to predict y.
use given model to predict y.
use given model to predict y.


In [12]:
co_obj1.compute_cohort_shapley()

compute Shapley values based on permutations
parallel processing with 4 processes
use given y values instead of model prediction.use given y values instead of model prediction.use given y values instead of model prediction.

use given y values instead of model prediction.
compute Shapley values based on permutationscompute Shapley values based on permutations
compute Shapley values based on permutations


compute Shapley values based on permutations

  0%|          | 0/262 [00:00<?, ?it/s]

100%|██████████| 262/262 [01:13<00:00,  3.56it/s]


In [13]:
co_obj2.compute_cohort_shapley()

compute Shapley values based on permutations
parallel processing with 4 processes
use given y values instead of model prediction.use given y values instead of model prediction.use given y values instead of model prediction.
compute Shapley values based on permutations



  0%|          | 0/262 [00:00<?, ?it/s]


use given y values instead of model prediction.compute Shapley values based on permutationscompute Shapley values based on permutations


compute Shapley values based on permutations


100%|██████████| 261/261 [01:15<00:00,  3.44it/s]


In [14]:
co_obj3.compute_cohort_shapley()

compute Shapley values based on permutations
parallel processing with 4 processes
use given y values instead of model prediction.use given y values instead of model prediction.use given y values instead of model prediction.use given y values instead of model prediction.

compute Shapley values based on permutationscompute Shapley values based on permutations


compute Shapley values based on permutations


  0%|          | 0/261 [00:00<?, ?it/s]


compute Shapley values based on permutations

  0%|          | 0/262 [00:00<?, ?it/s]

100%|██████████| 261/261 [00:37<00:00,  6.94it/s]


In [15]:
#they match as expected (up to some floating point uncertainty after 11 digits)
np.product(np.round(cs_obj.shapley_values,11) == np.round(co_obj1.shapley_values,11))

1

In [16]:
#same with 2
np.product(np.round(cs_obj.shapley_values,11) == np.round(co_obj2.shapley_values,11))

1

In [17]:
co_obj1.shapley_values

array([[ 0.11056243,  0.22988071,  0.00861183,  0.00919177, -0.00825117,
         0.18718278],
       [ 0.11729806, -0.17132534,  0.13500118,  0.0239048 ,  0.07535806,
         0.162311  ],
       [ 0.09335632,  0.18622282,  0.06834582,  0.0255391 ,  0.0563203 ,
         0.13319321],
       ...,
       [-0.08420768, -0.14395155,  0.00511031, -0.00567139, -0.01718519,
        -0.04297643],
       [-0.08378296, -0.14012262, -0.00202045, -0.00588476, -0.01707572,
        -0.04218951],
       [-0.08829549, -0.12763629, -0.01537655, -0.00640022, -0.0173368 ,
        -0.04269057]])

In [18]:
#We can see structure 3 is different as expected
co_obj3.shapley_values

array([[ 0.10592807,  0.2520752 , -0.00139914,  0.00706344, -0.00903532,
         0.1825461 ],
       [ 0.11781263, -0.19230006,  0.12263129,  0.04425368,  0.0837949 ,
         0.16635531],
       [ 0.07606748,  0.19206173,  0.06616942,  0.04451975,  0.06648068,
         0.11767849],
       ...,
       [-0.07794238, -0.15137018,  0.00432093, -0.00804722, -0.0192649 ,
        -0.03657818],
       [-0.07731778, -0.14774367, -0.00309841, -0.00818168, -0.01911756,
        -0.03561693],
       [-0.08205516, -0.13594112, -0.0157298 , -0.00862228, -0.01901619,
        -0.03637137]])

In [19]:
mc_test_num = 500
cs_mc_obj = cs.CohortShapley(f, similarity.similar_in_distance_cutoff, subject_id, X.values, parallel=4,mc_num = mc_test_num)
co_mc_obj1 = co.CohortOwen(titanic_union_structure1,f, similarity.similar_in_distance_cutoff, subject_id, X.values, parallel=4, mc_num = mc_test_num)
co_mc_obj3 = co.CohortOwen(titanic_union_structure3,f, similarity.similar_in_distance_cutoff, subject_id, X.values, parallel=4, mc_num = mc_test_num)

use given model to predict y.
use given model to predict y.
use given model to predict y.


In [20]:
cs_mc_obj.compute_cohort_shapley()

compute Shapley values based on permutations
parallel processing with 4 processes
use given y values instead of model prediction.use given y values instead of model prediction.
compute Shapley values based on permutations

compute Shapley values based on permutationsuse given y values instead of model prediction.use given y values instead of model prediction.




  0%|          | 0/261 [00:00<?, ?it/s]

compute Shapley values based on permutations


  0%|          | 0/262 [00:00<?, ?it/s]

compute Shapley values based on permutations


100%|██████████| 261/261 [01:07<00:00,  3.85it/s]


In [21]:
co_mc_obj1.compute_cohort_shapley()

compute Shapley values based on permutations
parallel processing with 4 processes
use given y values instead of model prediction.
compute Shapley values based on permutationsuse given y values instead of model prediction.
use given y values instead of model prediction.use given y values instead of model prediction.


compute Shapley values based on permutationscompute Shapley values based on permutations

  0%|          | 0/262 [00:00<?, ?it/s]



compute Shapley values based on permutations


100%|██████████| 262/262 [01:04<00:00,  4.08it/s]


In [22]:
co_mc_obj3.compute_cohort_shapley()

compute Shapley values based on permutations
parallel processing with 4 processes
use given y values instead of model prediction.use given y values instead of model prediction.

compute Shapley values based on permutationsuse given y values instead of model prediction.use given y values instead of model prediction.compute Shapley values based on permutations



compute Shapley values based on permutations

  0%|          | 0/261 [00:00<?, ?it/s]


compute Shapley values based on permutations

  0%|          | 0/262 [00:00<?, ?it/s]

100%|██████████| 261/261 [00:51<00:00,  5.06it/s]


In [23]:
cs_mc_obj.shapley_values

array([[ 0.10337342,  0.23100896,  0.00845672,  0.00972722, -0.00893005,
         0.19354207],
       [ 0.10719047, -0.17105392,  0.13498999,  0.02265206,  0.0748674 ,
         0.17390177],
       [ 0.0866791 ,  0.18616559,  0.07004549,  0.02348711,  0.05564852,
         0.14095174],
       ...,
       [-0.0809341 , -0.14384591,  0.00504767, -0.00617077, -0.01700487,
        -0.04597393],
       [-0.08057668, -0.13998576, -0.00219006, -0.00638664, -0.01689608,
        -0.04504081],
       [-0.08521219, -0.12758574, -0.01519743, -0.00690126, -0.01708687,
        -0.04575245]])

In [25]:
co_mc_obj1.shapley_values

array([[ 0.10960311,  0.23570953,  0.00744752,  0.00906646, -0.00958393,
         0.18493566],
       [ 0.11662762, -0.17489011,  0.13025553,  0.02734046,  0.08012109,
         0.16309316],
       [ 0.09224697,  0.18725324,  0.06497299,  0.02953815,  0.06078496,
         0.12818124],
       ...,
       [-0.08286058, -0.14512938,  0.00508072, -0.00595833, -0.01846278,
        -0.04155158],
       [-0.08243185, -0.14137288, -0.00207618, -0.00612243, -0.0183703 ,
        -0.0407024 ],
       [-0.08689716, -0.12925665, -0.01516053, -0.00658196, -0.01853701,
        -0.04130261]])

In [32]:
print(np.mean(np.abs(cs_obj.shapley_values),axis=0))
print(np.mean(np.abs(co_obj3.shapley_values),axis=0))
print(np.mean(np.abs(cs_mc_obj.shapley_values),axis=0))
print(np.mean(np.abs(co_mc_obj1.shapley_values),axis=0))
print(np.mean(np.abs(co_mc_obj3.shapley_values),axis=0))

[0.08878474 0.20438473 0.03683341 0.01775961 0.02689778 0.04904305]
[0.08540378 0.20615329 0.03287472 0.02121614 0.02960778 0.04636687]
[0.08583624 0.20402881 0.03590333 0.01872919 0.0265624  0.05236259]
[0.08794271 0.2051366  0.03614543 0.0177867  0.02854659 0.04845058]
[0.08615576 0.20151165 0.03283453 0.02180425 0.03088409 0.04802927]


In [31]:
co_mc_obj3.permutations

array([[2, 1, 3, 4, 0, 5],
       [1, 5, 0, 2, 3, 4],
       [5, 0, 3, 4, 1, 2],
       ...,
       [2, 1, 0, 5, 4, 3],
       [1, 5, 0, 2, 3, 4],
       [1, 2, 4, 3, 5, 0]])